In [3]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 33.4 MB/s eta 0:00:00


In [130]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch

In [131]:
def clean(path):
    df = pd.read_csv(path)
    df['time'] = pd.to_datetime(df['time'])
    return df

df = clean("train.csv")
df.head

<bound method NDFrame.head of                      time       low      high      open     close       volume
0     2015-11-30 12:00:00    367.50    379.42    378.16    373.00  3031.105717
1     2015-11-30 18:00:00    372.25    378.55    373.00    376.86  2652.465161
2     2015-12-01 00:00:00    375.80    378.94    376.88    378.01  1695.388592
3     2015-12-01 06:00:00    354.60    378.54    378.00    361.20  2682.538466
4     2015-12-01 12:00:00    355.00    364.86    361.21    360.14  2864.005011
...                   ...       ...       ...       ...       ...          ...
11622 2023-11-14 06:00:00  36249.50  36756.16  36642.00  36272.73  1455.611439
11623 2023-11-14 12:00:00  35855.00  36708.43  36272.63  36088.57  5289.680943
11624 2023-11-14 18:00:00  34758.64  36122.76  36090.42  35554.09  7511.702326
11625 2023-11-15 00:00:00  35358.45  35670.90  35554.10  35610.51  1929.755005
11626 2023-11-15 06:00:00  35533.29  36293.22  35610.08  36253.78  1693.626865

[11627 rows x 6 colum

In [132]:
def normalize(df):
    scaler = MinMaxScaler()
    cols = ['low', 'high', 'open', 'close', 'volume']  # adjust with your column names
    df[cols] = scaler.fit_transform(df[cols])
    return df, scaler

def reverse_norm(pred, scaler):
    predictions = pred.reshape(-1, 1)
    real_pred = scaler.inverse_transform(predictions)
    return real_pred
    
norm, scaler = normalize(df)
norm.head

<bound method NDFrame.head of                      time       low      high      open     close    volume
0     2015-11-30 12:00:00  0.005457  0.000341  0.000368  0.000293  0.029584
1     2015-11-30 18:00:00  0.005527  0.000328  0.000293  0.000349  0.025770
2     2015-12-01 00:00:00  0.005580  0.000334  0.000350  0.000366  0.016130
3     2015-12-01 06:00:00  0.005265  0.000328  0.000366  0.000120  0.026073
4     2015-12-01 12:00:00  0.005271  0.000129  0.000120  0.000104  0.027901
...                   ...       ...       ...       ...       ...       ...
11622 2023-11-14 06:00:00  0.538342  0.530274  0.531431  0.526014  0.013715
11623 2023-11-14 12:00:00  0.532483  0.529579  0.526022  0.523317  0.052333
11624 2023-11-14 18:00:00  0.516201  0.521047  0.523353  0.515490  0.074715
11625 2023-11-15 00:00:00  0.525109  0.514464  0.515499  0.516316  0.018491
11626 2023-11-15 06:00:00  0.527706  0.523530  0.516319  0.525737  0.016112

[11627 rows x 6 columns]>

In [133]:
def create_sequences(df, window_size):
    sequences = []
    df_size = len(df)
    for i in range(df_size - window_size):
        # Here, i: start of the sequence, i + window_size: end of the sequence
        sequence = df[i:i + window_size]
        label = df[i + window_size:i + window_size + 1]  # next value to be predicted
        sequences.append((sequence, label))
    return sequences

window_size = 10  # This is just an example value

# Assuming 'close' is what you want to predict
sequences = create_sequences(norm['close'].values, window_size)
sequences = [(torch.FloatTensor(seq), torch.FloatTensor(lbl)) for seq, lbl in sequences]
#print(sequences[:10])

In [134]:
def split_sequences(sequences, test_size=0.4, val_size=0.5, random_state=42):
    X, y = zip(*sequences)  # No change here, but X and y are already tensors
    X = torch.stack(X)  # Stack all sequence tensors
    y = torch.stack(y).squeeze()  # Stack all label tensors and remove extra dimension

    # Shuffle the data
    indices = torch.randperm(X.size(0))
    X, y = X[indices], y[indices]

    # Calculate split sizes
    train_end = int(X.size(0) * (1 - test_size))
    val_end = int(train_end * (1 - val_size))

    # Split the data
    X_train, X_temp = X[:train_end], X[train_end:]
    y_train, y_temp = y[:train_end], y[train_end:]
    X_val, X_test = X_temp[:val_end], X_temp[val_end:]
    y_val, y_test = y_temp[:val_end], y_temp[val_end:]

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = split_sequences(sequences)

In [135]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Assuming you have already run the modified split_sequences function
# and have X_train, X_val, X_test, y_train, y_val, y_test

# Create TensorDataset
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

# Create DataLoaders
batch_size = 64  # Adjust as necessary
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [136]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)

        # The output layer that maps the hidden state output to the desired output size
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), -1, input_size))
        predictions = self.linear(lstm_out[:, -1, :])
        return predictions

# Instantiate the model, loss function, and optimizer
num_features = 5
input_size = num_features  # number of features in your input
hidden_layer_size = 180  # number of features in hidden state
output_size = 1  # predict one feature

model = LSTM(input_size, hidden_layer_size, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [145]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        # Setting bidirectional=True doubles the output feature dimension 
        # because it concatenates the hidden states from both directions
        self.lstm = nn.LSTM(input_size, hidden_layer_size, 
                            batch_first=True, bidirectional=True)

        # Since the LSTM is bidirectional, we need to double the input feature dimension
        self.linear = nn.Linear(hidden_layer_size * 2, output_size)

    def forward(self, input_seq):
        # lstm_out shape is (batch, seq_len, num_directions * hidden_size)
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), -1, input_size))
        
        # We take the last time step's output from both directions
        predictions = self.linear(lstm_out[:, -1, :])
        return predictions

#hidden_layer_size = 180  # number of features in hidden state

bi_model = BiLSTM(input_size, hidden_layer_size, output_size)

bi_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [138]:
# training for standard LSTM

epochs = 500  # This is low, typically you'd have more epochs

for epoch in range(epochs):
    # Training
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(inputs)
        labels = labels.view_as(y_pred)
        single_loss = criterion(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            y_pred = model(inputs)
            val_loss += criterion(y_pred, labels).item()
    val_loss /= len(val_loader)

    print(f'Epoch {epoch+1} train_loss: {single_loss.item():.10f} val_loss: {val_loss:.10f}')


/Users/loganwork/anaconda3/envs/crypto_model/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/loganwork/anaconda3/envs/crypto_model/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 train_loss: 0.0316158012 val_loss: 0.0620016681
Epoch 2 train_loss: 0.0069178897 val_loss: 0.0821871558
Epoch 3 train_loss: 0.0005344134 val_loss: 0.1052439942
Epoch 4 train_loss: 0.0004439712 val_loss: 0.1118299652
Epoch 5 train_loss: 0.0002851372 val_loss: 0.1123840959
Epoch 6 train_loss: 0.0002024353 val_loss: 0.1125983956
Epoch 7 train_loss: 0.0002543227 val_loss: 0.1126371359
Epoch 8 train_loss: 0.0003676412 val_loss: 0.1127246758
Epoch 9 train_loss: 0.0002416756 val_loss: 0.1128042874
Epoch 10 train_loss: 0.0000852508 val_loss: 0.1126651916
Epoch 11 train_loss: 0.0002522433 val_loss: 0.1125624926
Epoch 12 train_loss: 0.0002764882 val_loss: 0.1127866089
Epoch 13 train_loss: 0.0001995529 val_loss: 0.1130222933
Epoch 14 train_loss: 0.0003042336 val_loss: 0.1126736757
Epoch 15 train_loss: 0.0002719340 val_loss: 0.1128663283
Epoch 16 train_loss: 0.0000792651 val_loss: 0.1125529106
Epoch 17 train_loss: 0.0002729457 val_loss: 0.1130317332
Epoch 18 train_loss: 0.0001249508 val_lo

Epoch 145 train_loss: 0.0001071306 val_loss: 0.1125924102
Epoch 146 train_loss: 0.0000389395 val_loss: 0.1134597375
Epoch 147 train_loss: 0.0001117307 val_loss: 0.1134924374
Epoch 148 train_loss: 0.0000494746 val_loss: 0.1133075263
Epoch 149 train_loss: 0.0000492210 val_loss: 0.1136605695
Epoch 150 train_loss: 0.0000358955 val_loss: 0.1127418847
Epoch 151 train_loss: 0.0000460372 val_loss: 0.1130085929
Epoch 152 train_loss: 0.0000738064 val_loss: 0.1135736992
Epoch 153 train_loss: 0.0000548319 val_loss: 0.1141210496
Epoch 154 train_loss: 0.0000872358 val_loss: 0.1119706270
Epoch 155 train_loss: 0.0000371082 val_loss: 0.1125231313
Epoch 156 train_loss: 0.0000231411 val_loss: 0.1130766126
Epoch 157 train_loss: 0.0000329636 val_loss: 0.1138247960
Epoch 158 train_loss: 0.0000732225 val_loss: 0.1135207990
Epoch 159 train_loss: 0.0000596370 val_loss: 0.1128636194
Epoch 160 train_loss: 0.0000408327 val_loss: 0.1134204657
Epoch 161 train_loss: 0.0001706966 val_loss: 0.1129085190
Epoch 162 trai

Epoch 287 train_loss: 0.0000794272 val_loss: 0.1128388962
Epoch 288 train_loss: 0.0000587468 val_loss: 0.1119338992
Epoch 289 train_loss: 0.0000255186 val_loss: 0.1130715600
Epoch 290 train_loss: 0.0000266363 val_loss: 0.1135104534
Epoch 291 train_loss: 0.0000283974 val_loss: 0.1127764969
Epoch 292 train_loss: 0.0000407512 val_loss: 0.1130050436
Epoch 293 train_loss: 0.0000301068 val_loss: 0.1130920696
Epoch 294 train_loss: 0.0000534258 val_loss: 0.1136867010
Epoch 295 train_loss: 0.0001449427 val_loss: 0.1131442302
Epoch 296 train_loss: 0.0000395840 val_loss: 0.1136048872
Epoch 297 train_loss: 0.0000597724 val_loss: 0.1137127077
Epoch 298 train_loss: 0.0000829167 val_loss: 0.1128329311
Epoch 299 train_loss: 0.0000175883 val_loss: 0.1127230283
Epoch 300 train_loss: 0.0000348516 val_loss: 0.1132898267
Epoch 301 train_loss: 0.0000361573 val_loss: 0.1130856553
Epoch 302 train_loss: 0.0000645093 val_loss: 0.1126067233
Epoch 303 train_loss: 0.0000519385 val_loss: 0.1131906986
Epoch 304 trai

Epoch 429 train_loss: 0.0000118656 val_loss: 0.1123911489
Epoch 430 train_loss: 0.0000178100 val_loss: 0.1131179688
Epoch 431 train_loss: 0.0000374779 val_loss: 0.1129065006
Epoch 432 train_loss: 0.0000408917 val_loss: 0.1129593590
Epoch 433 train_loss: 0.0000116737 val_loss: 0.1132803836
Epoch 434 train_loss: 0.0000138911 val_loss: 0.1131741433
Epoch 435 train_loss: 0.0000208323 val_loss: 0.1130250741
Epoch 436 train_loss: 0.0000752475 val_loss: 0.1126197520
Epoch 437 train_loss: 0.0000496523 val_loss: 0.1133167752
Epoch 438 train_loss: 0.0000307733 val_loss: 0.1130843276
Epoch 439 train_loss: 0.0000463752 val_loss: 0.1129454538
Epoch 440 train_loss: 0.0000312344 val_loss: 0.1130785373
Epoch 441 train_loss: 0.0000329663 val_loss: 0.1135615173
Epoch 442 train_loss: 0.0000453808 val_loss: 0.1135539762
Epoch 443 train_loss: 0.0000673871 val_loss: 0.1127085268
Epoch 444 train_loss: 0.0000286970 val_loss: 0.1134091808
Epoch 445 train_loss: 0.0000588041 val_loss: 0.1133203986
Epoch 446 trai

In [146]:
# training for Bi-LSTM

for epoch in range(epochs):
    # Training
    bi_model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(inputs)
        labels = labels.view_as(y_pred)
        single_loss = criterion(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    # Validation
    bi_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            y_pred = model(inputs)
            val_loss += criterion(y_pred, labels).item()
    val_loss /= len(val_loader)

    print(f'Epoch {epoch+1} train_loss: {single_loss.item():.10f} val_loss: {val_loss:.10f}')

/Users/loganwork/anaconda3/envs/crypto_model/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/loganwork/anaconda3/envs/crypto_model/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 train_loss: 0.0000303562 val_loss: 0.1128592911
Epoch 2 train_loss: 0.0000213959 val_loss: 0.1127702783
Epoch 3 train_loss: 0.0000640498 val_loss: 0.1129590200
Epoch 4 train_loss: 0.0000185229 val_loss: 0.1135864391
Epoch 5 train_loss: 0.0000662308 val_loss: 0.1128986334
Epoch 6 train_loss: 0.0000256227 val_loss: 0.1134182930
Epoch 7 train_loss: 0.0000200374 val_loss: 0.1134850560
Epoch 8 train_loss: 0.0000191856 val_loss: 0.1144921553
Epoch 9 train_loss: 0.0000222398 val_loss: 0.1133078688
Epoch 10 train_loss: 0.0000702690 val_loss: 0.1132656295
Epoch 11 train_loss: 0.0000598497 val_loss: 0.1136993747
Epoch 12 train_loss: 0.0000453049 val_loss: 0.1139200297
Epoch 13 train_loss: 0.0000293631 val_loss: 0.1134562693
Epoch 14 train_loss: 0.0000174231 val_loss: 0.1127772885
Epoch 15 train_loss: 0.0000870790 val_loss: 0.1135870725
Epoch 16 train_loss: 0.0000271910 val_loss: 0.1124069034
Epoch 17 train_loss: 0.0000288827 val_loss: 0.1135820561
Epoch 18 train_loss: 0.0000618623 val_lo

Epoch 145 train_loss: 0.0000486862 val_loss: 0.1126988713
Epoch 146 train_loss: 0.0000279840 val_loss: 0.1136871686
Epoch 147 train_loss: 0.0000368897 val_loss: 0.1135617509
Epoch 148 train_loss: 0.0000382988 val_loss: 0.1133388689
Epoch 149 train_loss: 0.0000852965 val_loss: 0.1132500482
Epoch 150 train_loss: 0.0000383389 val_loss: 0.1129043463
Epoch 151 train_loss: 0.0000291873 val_loss: 0.1129246515
Epoch 152 train_loss: 0.0000812634 val_loss: 0.1130787512
Epoch 153 train_loss: 0.0000284790 val_loss: 0.1137065623
Epoch 154 train_loss: 0.0000845811 val_loss: 0.1130117131
Epoch 155 train_loss: 0.0000786131 val_loss: 0.1134896274
Epoch 156 train_loss: 0.0000162275 val_loss: 0.1128578204
Epoch 157 train_loss: 0.0000192513 val_loss: 0.1132501468
Epoch 158 train_loss: 0.0000941601 val_loss: 0.1131697019
Epoch 159 train_loss: 0.0000415193 val_loss: 0.1130925815
Epoch 160 train_loss: 0.0000249289 val_loss: 0.1123735031
Epoch 161 train_loss: 0.0000162950 val_loss: 0.1130256081
Epoch 162 trai

Epoch 287 train_loss: 0.0000443852 val_loss: 0.1134607306
Epoch 288 train_loss: 0.0000395676 val_loss: 0.1126946804
Epoch 289 train_loss: 0.0000320514 val_loss: 0.1136373147
Epoch 290 train_loss: 0.0000175913 val_loss: 0.1130173943
Epoch 291 train_loss: 0.0000575636 val_loss: 0.1135580110
Epoch 292 train_loss: 0.0000824633 val_loss: 0.1133471131
Epoch 293 train_loss: 0.0000201892 val_loss: 0.1133541981
Epoch 294 train_loss: 0.0000686329 val_loss: 0.1136529205
Epoch 295 train_loss: 0.0000167777 val_loss: 0.1128197255
Epoch 296 train_loss: 0.0000187607 val_loss: 0.1125279432
Epoch 297 train_loss: 0.0000190627 val_loss: 0.1132606749
Epoch 298 train_loss: 0.0000259911 val_loss: 0.1134939740
Epoch 299 train_loss: 0.0000426764 val_loss: 0.1135088814
Epoch 300 train_loss: 0.0000310807 val_loss: 0.1128562286
Epoch 301 train_loss: 0.0000950317 val_loss: 0.1134852147
Epoch 302 train_loss: 0.0000132017 val_loss: 0.1129333231
Epoch 303 train_loss: 0.0000445164 val_loss: 0.1120873535
Epoch 304 trai

Epoch 429 train_loss: 0.0000357993 val_loss: 0.1131759234
Epoch 430 train_loss: 0.0000246265 val_loss: 0.1129685237
Epoch 431 train_loss: 0.0000427490 val_loss: 0.1128790053
Epoch 432 train_loss: 0.0000475187 val_loss: 0.1129786475
Epoch 433 train_loss: 0.0000460389 val_loss: 0.1133520432
Epoch 434 train_loss: 0.0000139612 val_loss: 0.1129440581
Epoch 435 train_loss: 0.0000382416 val_loss: 0.1132571462
Epoch 436 train_loss: 0.0000239989 val_loss: 0.1138019529
Epoch 437 train_loss: 0.0000457348 val_loss: 0.1130309385
Epoch 438 train_loss: 0.0000351062 val_loss: 0.1132676726
Epoch 439 train_loss: 0.0001788344 val_loss: 0.1131688115
Epoch 440 train_loss: 0.0000446386 val_loss: 0.1132474318
Epoch 441 train_loss: 0.0000766489 val_loss: 0.1128010576
Epoch 442 train_loss: 0.0000289588 val_loss: 0.1126022774
Epoch 443 train_loss: 0.0000286189 val_loss: 0.1134549677
Epoch 444 train_loss: 0.0000451036 val_loss: 0.1134876641
Epoch 445 train_loss: 0.0000564353 val_loss: 0.1133746178
Epoch 446 trai

In [140]:
# LSTM eval
model.eval()
test_loss = 0.0
predictions = []
actuals = []

with torch.no_grad():
    for inputs, labels in test_loader:
        # Forward pass
        y_pred = model(inputs)
        labels = labels.view_as(y_pred)

        # Calculate the batch loss
        loss = criterion(y_pred, labels)
        test_loss += loss.item()

        # Store predictions and actual values for further analysis if needed
        predictions.append(y_pred.numpy())  # or y_pred.cpu().numpy() if using GPU
        actuals.append(labels.numpy())

# Calculate average loss over the test set
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.10f}')
# Test Loss: 0.0004203604

Test Loss: 0.0000433352


In [147]:
# bi-LSTM eval
bi_model.eval()
bi_test_loss = 0.0
bi_predictions = []
bi_actuals = []

with torch.no_grad():
    for inputs, labels in test_loader:
        # Forward pass
        y_pred = bi_model(inputs)
        labels = labels.view_as(y_pred)

        # Calculate the batch loss
        loss = criterion(y_pred, labels)
        test_loss += loss.item()

        # Store predictions and actual values for further analysis if needed
        bi_predictions.append(y_pred.numpy())  # or y_pred.cpu().numpy() if using GPU
        bi_actuals.append(labels.numpy())

# Calculate average loss over the test set
bi_test_loss /= len(test_loader)
print(f'Test Loss: {bi_test_loss:.10f}')
# Test Loss: 0.0004203604

Test Loss: 0.0000000000


In [142]:
# inference for LSTM
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Load and normalize the inference data
inf_data = pd.read_csv("inf.csv")
print(inf_data.tail)

# Fit a separate scaler for the 'close' feature
close_scaler = MinMaxScaler()
inf_data['close'] = close_scaler.fit_transform(inf_data[['close']])

# Prepare the initial recent data for prediction
window_size = 10  # Adjust based on your model's training
recent_data = inf_data['close'][-window_size:].values.reshape(1, window_size, 1)  # Reshape for single batch, single feature

# Recursively predict next 5 time steps
num_predictions = 16
predictions = []
bi_predictions = []

for _ in range(num_predictions):
    # Convert recent data to tensor
    recent_data_tensor = torch.FloatTensor(recent_data)

    # Perform inference
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        predicted_next_step = model(recent_data_tensor).numpy()  # Get the model's prediction for the next step

    # Inverse transform to get actual price scale for the predicted step
    predicted_close_price = close_scaler.inverse_transform(predicted_next_step.reshape(-1, 1))[0, 0]
    predictions.append(predicted_close_price)

    # Update the recent data with the predicted value
    recent_data = np.roll(recent_data, -1, axis=1)  # Shift everything one step to the left
    recent_data[0, -1, 0] = predicted_next_step  # Insert the prediction at the end of recent_data

# predictions now contains the 5 sequential forecasted values
print("Predicted next 5 closing prices:", predictions)


<bound method NDFrame.tail of                     time       low      high      open     close       volume
0    2023-11-16 12:00:00  36260.32  37332.32  37260.42  36316.17  6840.344833
1    2023-11-16 18:00:00  35511.11  36400.18  36314.57  36161.15  7177.317828
2    2023-11-17 00:00:00  36097.28  36674.75  36161.15  36363.27  2676.160827
3    2023-11-17 06:00:00  36131.00  36473.75  36365.33  36382.68  1550.020928
4    2023-11-17 12:00:00  35869.00  36831.99  36382.69  36508.83  4979.856742
..                   ...       ...       ...       ...       ...          ...
151  2023-12-24 06:00:00  43434.62  43721.81  43517.85  43655.99   637.606020
152  2023-12-24 12:00:00  43578.33  43901.70  43658.10  43654.11   882.832758
153  2023-12-24 18:00:00  42614.17  43719.56  43656.52  43025.03  2143.310127
154  2023-12-25 00:00:00  42755.35  43241.15  43025.02  43231.21  1245.264334
155  2023-12-25 06:00:00  43056.87  43339.64  43233.08  43188.37   807.055913

[156 rows x 6 columns]>
Predicted

/var/folders/0x/0_w4ycb174s35r457mw7vlb40000gn/T/ipykernel_68161/190836376.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  recent_data[0, -1, 0] = predicted_next_step  # Insert the prediction at the end of recent_data
/var/folders/0x/0_w4ycb174s35r457mw7vlb40000gn/T/ipykernel_68161/190836376.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  recent_data[0, -1, 0] = predicted_next_step  # Insert the prediction at the end of recent_data
/var/folders/0x/0_w4ycb174s35r457mw7vlb40000gn/T/ipykernel_68161/190836376.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure y

In [144]:
# inference for bi-LSTM
# Fit a separate scaler for the 'close' feature
bi_predictions = []

for _ in range(num_predictions):
    # Convert recent data to tensor
    recent_data_tensor = torch.FloatTensor(recent_data)

    # Perform inference
    bi_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        predicted_next_step = bi_model(recent_data_tensor).numpy()  # Get the model's prediction for the next step

    # Inverse transform to get actual price scale for the predicted step
    predicted_close_price = close_scaler.inverse_transform(predicted_next_step.reshape(-1, 1))[0, 0]
    bi_predictions.append(predicted_close_price)

    # Update the recent data with the predicted value
    recent_data = np.roll(recent_data, -1, axis=1)  # Shift everything one step to the left
    recent_data[0, -1, 0] = predicted_next_step  # Insert the prediction at the end of recent_data

# predictions now contains the 5 sequential forecasted values
print("Predicted next 5 closing prices:", bi_predictions)

Predicted next 5 closing prices: [35652.38, 35652.43, 35652.37, 35652.3, 35652.258, 35652.246, 35652.246, 35652.254, 35652.254, 35652.254, 35652.254, 35652.254, 35652.254, 35652.254, 35652.254, 35652.254]


/var/folders/0x/0_w4ycb174s35r457mw7vlb40000gn/T/ipykernel_68161/923226228.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  recent_data[0, -1, 0] = predicted_next_step  # Insert the prediction at the end of recent_data
/var/folders/0x/0_w4ycb174s35r457mw7vlb40000gn/T/ipykernel_68161/923226228.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  recent_data[0, -1, 0] = predicted_next_step  # Insert the prediction at the end of recent_data
/var/folders/0x/0_w4ycb174s35r457mw7vlb40000gn/T/ipykernel_68161/923226228.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure y